# 1
Inserire nuove features nel dataset che rappresentino:
- escursione termica gionarliera (MaxTemp-MinTemp)
- differenza di unimidità (Humidity3pm- Humidity 9am)
- vento medio (WindSpeed3pm,WindSpeed9am)

Eliminare l'attributo Location e trasformare gli attributi booleani in numerici. Dividere il dataset in train e test (20%). Calcolare che accuratezza si ottiene con un modello randomForest (100 alberi) e confrontarla con un dummyClassifier (strategia stratified)

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("Data/weather.csv",sep=";")
df

aggiungiamo le colonne:

In [ ]:
df['Escursione_Giornaliera'] = df['MaxTemp'] - df['MinTemp']
df['Diff_umidità'] = df['Humidity3pm'] - df['Humidity9am']
df['Vento_medio'] = (df['WindSpeed3pm'] + df['WindSpeed9am'])/2
df

In [ ]:
df = df.drop('Location',axis=1)
df

In [ ]:
from sklearn.preprocessing import LabelEncoder
y = df['RainTomorrow']
df = df.drop('RainTomorrow',axis=1)

lb = LabelEncoder()
y = lb.fit_transform(y)
y

In [ ]:
lb = LabelEncoder()
df['RainToday'] = lb.fit_transform(df['RainToday'])
df

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2,random_state=0)

classifiers = [RandomForestClassifier(n_estimators=100),DummyClassifier(strategy='stratified')]
names = ['RF','dummy']
for model,name in zip(classifiers,names):
    model.fit(X_train,y_train)
    pred = model.predict(X_test)
    score = accuracy_score(y_test,pred)
    print('accuracy of '+ name +' : ',score)

# 2 Verificare se i risultati migliorano dopo avere scalato i valori tra 0 e 1 e normalizzato con Normalizer
Per scalare da 0 a 1 si usa il MinMax_Scaler


In [ ]:
from sklearn.preprocessing import MinMaxScaler,Normalizer
scaler = MinMaxScaler(feature_range=(0,1))
normalize = Normalizer()
X = scaler.fit_transform(df)
X = normalize.fit_transform(X)
X

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=0)

for model,name in zip(classifiers,names):
    model.fit(X_train,y_train)
    pred = model.predict(X_test)
    score = accuracy_score(y_test,pred)
    print('accuracy of '+ name +' : ',score)

# 3 Nel dataset originale, il valore -1 per l'attributo Cloud3pm rappresenta un errore. Occorre individuare le istanze che assumono quel valore, usare il modello randomforest per stimare il valore di Cloud3pm. Sostituire il valore predetto a -1 e rieseguire tutte le normalizzazioni del punto precedente e valutare se si introduce un miglioramento. Valutare cosa succede se invece di effettuare la sostituzione dei valori si eliminano le istanze non corrette

In [ ]:
df

In [47]:
df_with_Tomorrow = df.copy()
df_with_Tomorrow['RainTomorrow'] = y
df_with_Tomorrow

,Month,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud3pm,Temp9am,Temp3pm,RainToday,Escursione_Giornaliera,Diff_umidità,Vento_medio,RainTomorrow
0,1,20.4,37.6,0.0,54,0,7,46,17,1013.4,1009.2,3,26.1,36.7,0,17.2,-29,3.5,0
1,1,20.9,33.6,0.4,50,9,17,54,30,1011.1,1008.4,8,24.8,31.7,0,12.7,-24,13.0,1
2,1,18.4,23.1,2.2,48,11,39,62,67,1014.0,1014.8,8,21.8,19.5,1,4.7,5,25.0,1
3,1,17.3,23.7,15.6,39,9,17,74,65,1017.9,1016.5,8,19.2,21.6,1,6.4,-9,13.0,1
4,1,15.5,22.9,6.8,31,6,9,92,63,1016.3,1013.9,8,17.2,22.2,1,7.4,-29,7.5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14463,12,22.1,35.8,63.8,43,17,31,77,41,997.8,995.2,2,26.0,33.5,1,13.7,-36,24.0,0
14464,12,22.6,36.8,0.0,50,30,15,63,39,1000.0,998.8,-1,29.7,34.0,0,14.2,-24,22.5,0
14465,12,23.2,38.0,0.0,33,17,17,25,14,1004.4,1001.0,-1,28.7,36.4,0,14.8,-11,17.0,0
14466,12,19.7,37.0,0.0,37,22,6,30,23,1004.6,1000.9,-1,28.2,35.1,0,17.3,-7,14.0,0


In [64]:
df_no_cloud  = df_with_Tomorrow[df_with_Tomorrow['Cloud3pm']  > 0]

In [65]:
#y_cloud = lb.fit_transform(y_cloud)
y_cloud = df_no_cloud ['Cloud3pm']
df_no_cloud = df_no_cloud.drop('Cloud3pm',axis=1)
df_no_cloud

,Month,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,Escursione_Giornaliera,Diff_umidità,Vento_medio,RainTomorrow
0,1,20.4,37.6,0.0,54,0,7,46,17,1013.4,1009.2,26.1,36.7,0,17.2,-29,3.5,0
1,1,20.9,33.6,0.4,50,9,17,54,30,1011.1,1008.4,24.8,31.7,0,12.7,-24,13.0,1
2,1,18.4,23.1,2.2,48,11,39,62,67,1014.0,1014.8,21.8,19.5,1,4.7,5,25.0,1
3,1,17.3,23.7,15.6,39,9,17,74,65,1017.9,1016.5,19.2,21.6,1,6.4,-9,13.0,1
4,1,15.5,22.9,6.8,31,6,9,92,63,1016.3,1013.9,17.2,22.2,1,7.4,-29,7.5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14459,12,24.8,32.6,0.0,44,20,11,59,57,1007.5,1004.0,28.2,30.0,0,7.8,-2,15.5,1
14460,12,22.9,29.9,21.8,35,13,19,85,58,1008.8,1005.1,24.6,29.3,1,7.0,-27,16.0,1
14461,12,22.8,29.5,9.2,54,19,26,96,75,1005.5,1001.1,24.3,28.0,1,6.7,-21,22.5,1
14462,12,22.1,27.4,83.8,72,31,43,100,100,994.6,989.2,22.2,24.2,1,5.3,0,37.0,1


In [66]:
X_train, X_test, y_train, y_test = train_test_split(df_no_cloud,y_cloud, test_size=0.2,random_state=0)

model = classifiers[0]
model.fit(X_train,y_train)
pred = model.predict(X_test)
pred

array([2, 7, 8, ..., 7, 8, 8])

In [68]:
media = pred.mean()

Poi sostituiamo il valore predetto con -1

In [69]:
df_replaced = df.replace(-1,media)
df_replaced

,Month,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud3pm,Temp9am,Temp3pm,RainToday,Escursione_Giornaliera,Diff_umidità,Vento_medio
0,1,20.4,37.6,0.0,54,0,7,46,17,1013.4,1009.2,3.000000,26.1,36.7,0,17.2,-29.0,3.5
1,1,20.9,33.6,0.4,50,9,17,54,30,1011.1,1008.4,8.000000,24.8,31.7,0,12.7,-24.0,13.0
2,1,18.4,23.1,2.2,48,11,39,62,67,1014.0,1014.8,8.000000,21.8,19.5,1,4.7,5.0,25.0
3,1,17.3,23.7,15.6,39,9,17,74,65,1017.9,1016.5,8.000000,19.2,21.6,1,6.4,-9.0,13.0
4,1,15.5,22.9,6.8,31,6,9,92,63,1016.3,1013.9,8.000000,17.2,22.2,1,7.4,-29.0,7.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14463,12,22.1,35.8,63.8,43,17,31,77,41,997.8,995.2,2.000000,26.0,33.5,1,13.7,-36.0,24.0
14464,12,22.6,36.8,0.0,50,30,15,63,39,1000.0,998.8,5.163164,29.7,34.0,0,14.2,-24.0,22.5
14465,12,23.2,38.0,0.0,33,17,17,25,14,1004.4,1001.0,5.163164,28.7,36.4,0,14.8,-11.0,17.0
14466,12,19.7,37.0,0.0,37,22,6,30,23,1004.6,1000.9,5.163164,28.2,35.1,0,17.3,-7.0,14.0


In [70]:
scaler = MinMaxScaler(feature_range=(0,1))
normalize = Normalizer()
X = scaler.fit_transform(df_replaced)
X = normalize.fit_transform(X)
X

array([[0.        , 0.34560832, 0.37728784, ..., 0.26725515, 0.15158887,
        0.02572417],
       [0.        , 0.33335104, 0.31374032, ..., 0.18673031, 0.15648378,
        0.09041285],
       [0.        , 0.27491876, 0.1823277 , ..., 0.06295316, 0.21145438,
        0.15839279],
       ...,
       [0.41027576, 0.33933471, 0.33835087, ..., 0.20376112, 0.17827459,
        0.11070933],
       [0.41447386, 0.30100119, 0.33157909, ..., 0.24061738, 0.18996719,
        0.0921053 ],
       [0.41012587, 0.34512033, 0.28759444, ..., 0.1279923 , 0.20018049,
        0.13345366]])

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=0)

for model,name in zip(classifiers,names):
    model.fit(X_train,y_train)
    pred = model.predict(X_test)
    score = accuracy_score(y_test,pred)
    print('accuracy of '+ name +' : ',score)

accuracy of RF :  0.8410504492052523
accuracy of dummy :  0.6482377332411887


In [72]:
df.loc[(df.Cloud3pm == -1)].index

Int64Index([    9,    11,    12,    14,    15,    17,    23,    29,    34,
               35,
            ...
            14449, 14453, 14454, 14455, 14456, 14457, 14464, 14465, 14466,
            14467],
           dtype='int64', length=6124)

In [73]:
df_eliminated = df.drop(df.loc[(df.Cloud3pm == -1)].index)
df_eliminated

,Month,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud3pm,Temp9am,Temp3pm,RainToday,Escursione_Giornaliera,Diff_umidità,Vento_medio
0,1,20.4,37.6,0.0,54,0,7,46,17,1013.4,1009.2,3,26.1,36.7,0,17.2,-29,3.5
1,1,20.9,33.6,0.4,50,9,17,54,30,1011.1,1008.4,8,24.8,31.7,0,12.7,-24,13.0
2,1,18.4,23.1,2.2,48,11,39,62,67,1014.0,1014.8,8,21.8,19.5,1,4.7,5,25.0
3,1,17.3,23.7,15.6,39,9,17,74,65,1017.9,1016.5,8,19.2,21.6,1,6.4,-9,13.0
4,1,15.5,22.9,6.8,31,6,9,92,63,1016.3,1013.9,8,17.2,22.2,1,7.4,-29,7.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14459,12,24.8,32.6,0.0,44,20,11,59,57,1007.5,1004.0,8,28.2,30.0,0,7.8,-2,15.5
14460,12,22.9,29.9,21.8,35,13,19,85,58,1008.8,1005.1,6,24.6,29.3,1,7.0,-27,16.0
14461,12,22.8,29.5,9.2,54,19,26,96,75,1005.5,1001.1,8,24.3,28.0,1,6.7,-21,22.5
14462,12,22.1,27.4,83.8,72,31,43,100,100,994.6,989.2,8,22.2,24.2,1,5.3,0,37.0


In [74]:
y_delete = np.delete(y,df.loc[(df.Cloud3pm == -1)].index)
len(y_delete)

8344

In [75]:
scaler = MinMaxScaler(feature_range=(0,1))
normalize = Normalizer()
X = scaler.fit_transform(df_eliminated)
X = normalize.fit_transform(X)
len(X)

8344

In [76]:
X_train, X_test, y_train, y_test = train_test_split(X, y_delete, test_size=0.2,random_state=0)

for model,name in zip(classifiers,names):
    model.fit(X_train,y_train)
    pred = model.predict(X_test)
    score = accuracy_score(y_test,pred)
    print('accuracy of '+ name +' : ',score)

accuracy of RF :  0.8358298382264829
accuracy of dummy :  0.6237267825044938


# 4 Realizzare una pipeline che effettui il punto 2 dell'esercizio
Proviamo con il Feature_Union

In [77]:
from sklearn.pipeline import FeatureUnion

scaler = MinMaxScaler(feature_range=(0,1))

normalize = Normalizer()

combine_features = FeatureUnion([('scale',scaler), ('norma',normalize)])
combine_features

FeatureUnion(transformer_list=[('scale', MinMaxScaler()),
                               ('norma', Normalizer())])

In [79]:
from sklearn.pipeline import Pipeline

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2,random_state=0)


for model,name in zip(classifiers,names):
    my_pipeline = Pipeline(steps=[('combined_features',combine_features),('model',model)],verbose=True)

    my_pipeline.fit(X_train,y_train)

    preds = my_pipeline.predict(X_test)

    score = accuracy_score(y_test,preds)
    print('accuracy '+ name+':',score)

[Pipeline] . (step 1 of 2) Processing combined_features, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   3.2s
accuracy RF: 0.8483068417415343
[Pipeline] . (step 1 of 2) Processing combined_features, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.0s
accuracy dummy: 0.6209398756046994


# 5 Realizzare una pipeline che effettui i punti 1 e 2 dell'esercizio

In [166]:
new_df = pd.read_csv("Data/weather.csv",sep=";")
new_df = new_df.drop('RainTomorrow',axis=1)
new_df

,Month,Location,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud3pm,Temp9am,Temp3pm,RainToday
0,1,Albury,20.4,37.6,0.0,54,0,7,46,17,1013.4,1009.2,3,26.1,36.7,No
1,1,Albury,20.9,33.6,0.4,50,9,17,54,30,1011.1,1008.4,8,24.8,31.7,No
2,1,Albury,18.4,23.1,2.2,48,11,39,62,67,1014.0,1014.8,8,21.8,19.5,Yes
3,1,Albury,17.3,23.7,15.6,39,9,17,74,65,1017.9,1016.5,8,19.2,21.6,Yes
4,1,Albury,15.5,22.9,6.8,31,6,9,92,63,1016.3,1013.9,8,17.2,22.2,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14463,12,Uluru,22.1,35.8,63.8,43,17,31,77,41,997.8,995.2,2,26.0,33.5,Yes
14464,12,Uluru,22.6,36.8,0.0,50,30,15,63,39,1000.0,998.8,-1,29.7,34.0,No
14465,12,Uluru,23.2,38.0,0.0,33,17,17,25,14,1004.4,1001.0,-1,28.7,36.4,No
14466,12,Uluru,19.7,37.0,0.0,37,22,6,30,23,1004.6,1000.9,-1,28.2,35.1,No


In [154]:
def dif(X,names):
    first = names[0]
    second = names[1]
    print(first,second)
    return (X[first] - X[second])

def mean(X,names):
    first = names[0]
    second = names[1]
    return  (X[first]+X[second])/2

def operations(op,X,names):
    switch={'-': dif(X, names),'m':mean(X, names)}
    return switch.get(op)

def Add_Feature(X,new_feature,column_names,op):
    X[new_feature] = operations(op,X,column_names)
    return X

def clean_data(X,names):
    for name in names:
        X = X.drop(name,axis=1)
    return X

def GetBoolean_column(X):
    categorical_cols = [cname for cname in X.columns if
                        X[cname].nunique() < 10 and
                        X[cname].dtype == "object"]
    return categorical_cols


def Ex_1(df):
    df['Escursione_Giornaliera'] = df['MaxTemp'] - df['MinTemp']
    df['Diff_umidità'] = df['Humidity3pm'] - df['Humidity9am']
    df['Vento_medio'] = (df['WindSpeed3pm'] + df['WindSpeed9am'])/2
    # remove Location
    return df.drop('Location',axis=1)

In [170]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

X_train, X_test, y_train, y_test = train_test_split(new_df, y, test_size=0.2,random_state=0)

bool_trasform = LabelEncoder()
indexes = new_df.columns.get_indexer(GetBoolean_column(new_df))

col_prep = ColumnTransformer(transformers=[('bool',bool_trasform,indexes)],remainder="passthrough")
func_ex_1 = FunctionTransformer(Ex_1, validate=False)

pre_process_pipeline = Pipeline(steps=[
        ('bool',col_prep),
        ('pre_Ex_1',func_ex_1),
        ('combined_features',combine_features),
        ('model',model)],verbose=True)

pre_process_pipeline.fit_transform(X_train,y_train)

# for model,name in zip(classifiers,names):
#     my_pipeline = Pipeline(steps=[('pre_processing',pre_process_pipeline),('combined_features',combine_features),('model',model)],verbose=True)
#
#     my_pipeline.fit(new_df,y)
#
#     preds = my_pipeline.predict(X_test)
#
#     score = accuracy_score(y_test,preds)
#     print('accuracy '+ name+':',score)

TypeError: fit_transform() takes 2 positional arguments but 3 were given

[12, 5, 1]